# ДЗ 3 
## Ранжирование и BM25

## Функция ранжирования BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Query$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $Doc$ запросу $Query$:

$$ BM25(Query, Doc) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,Doc)*(k+1)}{TF(q_i,Doc)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где    
$$$$
$\text{IDF}(q_i)$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в корпусе   
$n(q_i)$ — количество документов, содержащих слово $q_i$

>$TF(q_i,Doc)$ - частота слова $q_i$ в документе $Doc$    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75  
$l(d)$ - длина документа (количество слов в нём)   
$avgdl$ — средняя длина документа в корпусе    

### __Задача__:

Реализуйте поиск, где
- в качестве векторизации документов корпуса - слагаемые **BM25**
- формат хранения индекса - **матрица Document-Term**
- метрика близости пар (запрос, документ) - **BM25**

В реализации должно быть все то же, что во втором дз:
- функция индексации корпуса, на выходе которой посчитанная матрица Document-Term
- функция индексации запроса, на выходе которой посчитанный вектор запроса
- функция с реализацией подсчета близости запроса и документов корпуса, на выходе которой вектор, i-й элемент которого обозначает близость запроса с i-м документом корпуса. Сделать **обязательно векторно**.
- главная функция, объединяющая все это вместе; на входе - запрос, на выходе - отсортированные по убыванию имена документов коллекции

Обратите внимание:
- сортировку надо сделать **<font color='green'>обязательно векторно</font>** через маску; при не соблюдении минус два балла
- подсчет индекса надо сделать **<font color='green'>обязательно с использованием спарсованных матриц</font>**, то есть ни в какой момент времени векторизованный корпус не переводится в ndarray; при не соблюдении минус два балла
- надо сделать **<font color='green'>обязательно с учетом всех комментариев за свои предыдущие дз</font>**; при не соблюдении минус балл за каждый пункт
- использование хардкод локального пути => 0 баллов


В качестве корпуса возьмите корпус вопросов и ответов с Ответы Мейл) 👍😃 
[Ссылка для скачивания](https://www.kaggle.com/bobazooba/thousands-of-questions-about-love)

Описание структуры данных можно посмотреть по ссылке. В качестве документов корпуса берем значения из ключа *answers*, но не все, а один, у которого максимальный *value*. При этом ограничиваем количество документов до 50000. Пример ниже.


**На что направлена эта задача:** 
Реализация поисковика с использованием BM25.



In [1]:
import os
from pymystem3 import Mystem
from string import punctuation
punctuation += '...' + '—' + '…' + '«»'
import nltk
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
stopword = stopwords.words('russian')
from scipy import sparse
import json
m = Mystem()
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer 
from pprint import pprint

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trekc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Я пока не дропаю пустые строки, потому что хочу сохранить их индексы <br>
Я сделаю это после лемматизации, результат будет один

In [2]:
def first_processing(file):
    with open(file, 'r', encoding='utf-8') as f:
        corpus = list(f)[:50000]

    answers_corpus = []

    for part in corpus:
        d = dict()
        j_answers = json.loads(part)['answers']
        try: # бывает пустое поле answers
            for ind, gr_comments in enumerate(j_answers):

                try:
                    d[ind] = int(gr_comments['author_rating']['value'])

                except ValueError: # бывает пустое поле value
                    d[ind] = 0

            ind = sorted(d.items(), key=lambda item: item[1], reverse=True)[0][0]
            answers_corpus.append(j_answers[ind]['text'])

        except IndexError:
            pass
    
    return answers_corpus

In [3]:
answers_corpus = first_processing('questions_about_love.jsonl')

In [4]:
# Вопрос в том, стоит ли удалять пустые строки. 
#Тогда надо будет считать индексы выкинутых строк, чтобы в результате запроса выводить нужный текст

In [5]:
def second_processing(cleared_corpus):
    corpus = []
    dropped = []
    for text in cleared_corpus:
        text = re.sub('[0-9a-zA-Z]+', '', text)
        text = [word.lower().strip().strip(punctuation) for word in text.split()]
        #text = [x for x in text if x not in stopword]
        text = ' '.join([word for word in text if word != ''])
        corpus.append(text)
    
    # чтобы удалить пустые строки и сохранить их индексы
    # чтобы убрать их в изначальном датасете
    for ind, text in enumerate(corpus):
        if not text:
            dropped.append(ind)
            del corpus[ind]
        
    lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
    txtpart = lol(corpus, 1000)
    res = []
    for txtp in txtpart:
        alltexts = ' '.join([txt + ' br ' for txt in txtp])
        words = m.lemmatize(alltexts)
        doc = []
        for txt in words:
            if txt != '\n' and txt.strip() != '':
                if txt == 'br':
                    res.append(' '.join(doc))
                    doc = []
                else:
                    doc.append(txt)

    return res, dropped

In [6]:
cleared_corpus, dropped = second_processing(answers_corpus)

In [7]:
for ind in dropped:
    del answers_corpus[ind]

### Векторизация

In [8]:
def vectorization(cleared_corpus):
    count_vectorizer = CountVectorizer()
    tf_vectorizer = TfidfVectorizer(use_idf=False, norm='l2')
    tfidf_vectorizer = TfidfVectorizer(use_idf=True, norm='l2')
    
    x_count_vec = count_vectorizer.fit_transform(cleared_corpus) # для индексации запроса 
    x_tf_vec = tf_vectorizer.fit_transform(cleared_corpus) # матрица с tf
    x_tfidf_vec = tfidf_vectorizer.fit_transform(cleared_corpus) # матрица для idf
    
    idf = tfidf_vectorizer.idf_
    idf = np.expand_dims(idf, axis=0)
    tf = x_tf_vec

    values = []
    rows = []
    cols = []
    k = 2
    b = 0.75

    len_d = x_count_vec.sum(axis=1)
    avdl = len_d.mean()
    B_1 = (k * (1 - b + b * len_d / avdl))
    
    for i, j in zip(*tf.nonzero()): 
        rows.append(i)
        cols.append(j)
        A = idf[0][j] * tf[i,j] * k+1 
        B = tf[i,j] + B_1[i]
        AB = A / B

        values.append(np.asarray(AB)[0][0])
    
    return sparse.csr_matrix((values, (rows, cols))), count_vectorizer

In [9]:
sparced_matrix, count_vectorizer = vectorization(cleared_corpus)

### Индексирую запрос

In [10]:
def query_preprocessing(query, count_vectorizer):
    query = [word.lower().strip(punctuation).strip() for word in query.split()]
    query = m.lemmatize(' '.join([word for word in query]))
    query = ' '.join([word for word in query])
    query = ' '.join([word for word in query.split() if word != ''])
    query_vec = count_vectorizer.transform([query])
    return query_vec

In [11]:
def search(sparced_matrix, query_vec):
    scores = cosine_similarity(sparced_matrix, query_vec)
    sorted_scores_indx = np.argsort(scores, axis=0)[::-1]
    return list(np.array(answers_corpus)[sorted_scores_indx.ravel()][:10])

In [12]:
while True:
    query = input('Введите запрос (или "ОСТАНОВИТЕ" для остановки):')
    if query == 'ОСТАНОВИТЕ':
        break
    query_vec = query_preprocessing(query, count_vectorizer)
    pprint(search(sparced_matrix, query_vec))

Введите запрос (или "ОСТАНОВИТЕ" для остановки):мама мыла раму
['Мамам',
 'мама',
 'Мыть.',
 'Моя мама.',
 'Спроси у мамы',
 'Спроси у мамы',
 'У мамы спроси!',
 'Ваша мама и Вы?!',
 'Слушать маму',
 'Кому маме? Или тебе?']


KeyboardInterrupt: Interrupted by user

### __Задача__:

Реализуйте поиск, где
- в качестве векторизации документов корпуса - слагаемые **BM25**
- формат хранения индекса - **матрица Document-Term**
- метрика близости пар (запрос, документ) - **BM25**

В реализации должно быть все то же, что во втором дз:
- функция индексации корпуса, на выходе которой посчитанная матрица Document-Term
- функция индексации запроса, на выходе которой посчитанный вектор запроса
- функция с реализацией подсчета близости запроса и документов корпуса, на выходе которой вектор, i-й элемент которого обозначает близость запроса с i-м документом корпуса. Сделать **обязательно векторно**.
- главная функция, объединяющая все это вместе; на входе - запрос, на выходе - отсортированные по убыванию имена документов коллекции

Обратите внимание:
- сортировку надо сделать **<font color='green'>обязательно векторно</font>** через маску; при не соблюдении минус два балла
- подсчет индекса надо сделать **<font color='green'>обязательно с использованием спарсованных матриц</font>**, то есть ни в какой момент времени векторизованный корпус не переводится в ndarray; при не соблюдении минус два балла
- надо сделать **<font color='green'>обязательно с учетом всех комментариев за свои предыдущие дз</font>**; при не соблюдении минус балл за каждый пункт
- использование хардкод локального пути => 0 баллов


В качестве корпуса возьмите корпус вопросов и ответов с Ответы Мейл) 👍😃 
[Ссылка для скачивания](https://www.kaggle.com/bobazooba/thousands-of-questions-about-love)

Описание структуры данных можно посмотреть по ссылке. В качестве документов корпуса берем значения из ключа *answers*, но не все, а один, у которого максимальный *value*. При этом ограничиваем количество документов до 50000. Пример ниже.


**На что направлена эта задача:** 
Реализация поисковика с использованием BM25.

